<a href="https://colab.research.google.com/github/RohanDubeySF/Training/blob/master/Feature%20Engineering/Column-Transformer/ColumnTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('https://github.com/campusx-official/100-days-of-machine-learning/raw/refs/heads/main/day28-column-transformer/covid_toy.csv')
data.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [ ]:
data.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [ ]:
from sklearn.model_selection import train_test_split
x_train,X_test,y_train,y_test=train_test_split(data.drop(columns=['has_covid']),data['has_covid'],test_size=0.2)
x_train.shape

(80, 5)

In [ ]:
x_train.head(5)

,age,gender,fever,cough,city
67,65,Male,99.0,Mild,Bangalore
24,13,Female,100.0,Strong,Kolkata
83,17,Female,104.0,Mild,Kolkata
47,18,Female,104.0,Mild,Bangalore
17,40,Female,98.0,Strong,Delhi


# Without using Column Transformer

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder(categories=[['Mild','Strong']])


In [ ]:
cough=oe.fit_transform(x_train[['cough']])

In [ ]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()

In [ ]:
fever=si.fit_transform(x_train[['fever']])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(drop='first',sparse_output=False)

In [ ]:
gc=ohe.fit_transform(x_train[['gender','city']])

In [ ]:
data=np.concatenate((cough,fever,gc),axis=1)
data

array([[  0.        ,  99.        ,   1.        ,   0.        ,
          0.        ,   0.        ],
       [  1.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0.        ],
       [  0.        , 104.        ,   0.        ,   0.        ,
          1.        ,   0.        ],
       [  0.        , 104.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  1.        ,  98.        ,   0.        ,   1.        ,
          0.        ,   0.        ],
       [  0.        ,  98.        ,   0.        ,   1.        ,
          0.        ,   0.        ],
       [  1.        ,  98.        ,   0.        ,   0.        ,
          1.        ,   0.        ],
       [  0.        , 101.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        , 101.        ,   1.        ,   1.        ,
          0.        ,   0.        ],
       [  0.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0. 

In [ ]:
x_train_new=pd.DataFrame(data)
x_train_new.sample(5)

,0,1,2,3,4,5
20,1.0,100.866667,1.0,0.0,1.0,0.0
65,1.0,98.000000,1.0,0.0,0.0,0.0
59,1.0,100.866667,1.0,0.0,1.0,0.0
58,0.0,102.000000,0.0,0.0,0.0,0.0
44,0.0,102.000000,1.0,0.0,0.0,1.0


In [ ]:
x_train_new['Age']=x_train['age']
x_train_new.sample(5)


,0,1,2,3,4,5,Age
72,0.0,100.0,1.0,0.0,0.0,0.0,83.0
15,1.0,101.0,1.0,0.0,0.0,0.0,70.0
56,0.0,100.0,1.0,0.0,1.0,0.0,71.0
79,0.0,99.0,0.0,0.0,0.0,0.0,NaN
6,1.0,98.0,0.0,0.0,1.0,0.0,14.0


# Using Column Transformer


In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
transformer=ColumnTransformer(transformers=[
    ('tf1',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tf2',SimpleImputer(),['fever']),
    ('tf3', OneHotEncoder(drop='first',sparse_output=False), ['gender','city'])
], remainder='passthrough')

In [ ]:
x_train['city'].unique()

array(['Bangalore', 'Kolkata', 'Delhi', 'Mumbai'], dtype=object)

In [ ]:
transformer.fit_transform(x_train)

array([[  0.        ,  99.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  65.        ],
       [  1.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  13.        ],
       [  0.        , 104.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  17.        ],
       [  0.        , 104.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  18.        ],
       [  1.        ,  98.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  40.        ],
       [  0.        ,  98.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  80.        ],
       [  1.        ,  98.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [  0.        , 101.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  38.        ],
       [  0.        , 101.        ,   1.        ,   1.        ,
          0.    

In [ ]:
x_train_new2=pd.DataFrame(transformer.fit_transform(x_train),columns=['cough','fever','Male', 'Kolkata', 'Delhi', 'Mumbai','Age'])
x_train_new2.head(5)

,cough,fever,Male,Kolkata,Delhi,Mumbai,Age
0,0.0,99.0,1.0,0.0,0.0,0.0,65.0
1,1.0,100.0,0.0,0.0,1.0,0.0,13.0
2,0.0,104.0,0.0,0.0,1.0,0.0,17.0
3,0.0,104.0,0.0,0.0,0.0,0.0,18.0
4,1.0,98.0,0.0,1.0,0.0,0.0,40.0
